In [1]:
from HP_model import Problem
from mesa.batchrunner import BatchRunnerMP
import pandas as pd
from functools import partial

pd.set_option("precision", 3)


# Hong & Page - replication runs

The models in Hong & Page is run 50 times, with three different parameter combinations presented. Here the fourth permutation (large teams with many heuristics) is included, and the number of runs increased to 500. A 4-core 2.4 GHz laptop can complete approximately two runs per minute, so that this takes roughly 4 hours. 

In [3]:
fixed_params = {"n": 2000,
               "k": 3
               }
variable_params = {
                   "N_agents": (10, 20),
                   "l": (12, 20)
                   }

# l = 20 is much slower

batch_run = BatchRunnerMP(Problem,
                        4,
                        variable_parameters = variable_params,
                        fixed_parameters = fixed_params,
                        iterations=2,
                        max_steps=5,
                        model_reporters={"agent_descriptives": lambda m: m.agent_descriptives,
                        "best_solution": lambda m: m.best_solution})
batch_run.run_all()

8it [00:55,  7.00s/it]


({(10,
   12,
   2000,
   3,
   0): OrderedDict([('agent_descriptives',
                {'random': {'worst_agent': 83.6783421020347,
                  'average_agent': 85.08520358739457,
                  'top_agent': 86.35381954474913,
                  'team_average': 84.91754046388886,
                  'NPdiversity': 0.9259259259259259,
                  'Cdiversity': 0.9166666666666666},
                 'best': {'worst_agent': 83.6783421020347,
                  'average_agent': 85.08520358739457,
                  'top_agent': 86.35381954474913,
                  'team_average': 86.1884871198831,
                  'NPdiversity': 0.9185185185185185,
                  'Cdiversity': 0.6666666666666666}}),
               ('best_solution',
                {'random': 99.61644343925519, 'best': 98.96812987060643})]),
  (10,
   12,
   2000,
   3,
   1): OrderedDict([('agent_descriptives',
                {'random': {'worst_agent': 83.93870739060941,
                  'average_agent': 85

In [5]:
res = batch_run.get_model_vars_dataframe()

# Turn dictionaries into columns
res = pd.concat([res.drop(["best_solution"], axis = 1), res.best_solution.apply(pd.Series).add_suffix("_solution")], axis = 1)

def renamer(col, prefix):
    if col.endswith("agent"):
        return col
    else:
        return prefix + col

res_random = res.agent_descriptives.apply(pd.Series).random.apply(pd.Series).rename(mapper = partial(renamer, prefix = "random_"), axis = "columns")

res_best = res.agent_descriptives.apply(pd.Series).best.apply(pd.Series).rename(mapper = partial(renamer, prefix = "best_"), axis = "columns")

res = pd.concat([res.drop(["agent_descriptives"], axis=1), res_best, res_random[res_random.columns[pd.Series(res_random.columns).str.startswith('random_')]]], axis=1)

res["run_id"] = res.reset_index().index
res = res.rename(columns={"best_agent": "top_agent"})

#Pivot so that random and best groups can be easily compared
col_names = res.columns.values.tolist()

def check_var(col_name):
    return not(col_name.find("random_") != -1 or col_name.find("best_") != -1)


id_cols = list(filter(check_var, col_names))

out = pd.melt(res, id_cols)

out = out.join(out.variable.str.split("_", expand = True)).rename(columns={0:"team_type"}).pivot_table(index=id_cols + ["team_type"], columns=[1], values="value").reset_index()


In [29]:
#Performance and diversity of best versus random teams
out[["team_type", "N_agents", "l", "solution", "NPdiversity"]].groupby(["team_type", "N_agents", "l"]).describe().loc[:, (slice(None), ["mean", "std"])]


1                     solution         NPdiversity       
                          mean     std        mean    std
team_type N_agents l                                     
best      10       12   97.179   2.531       0.893  0.037
                   20   93.951   2.049       0.926  0.000
          20       12   93.016   6.995       0.872  0.015
                   20   95.780   2.576       0.901  0.014
random    10       12   96.647   4.199       0.915  0.016
                   20   96.428   2.241       0.959  0.037
          20       12   75.643  31.535       0.918  0.025
                   20   98.766   1.572       0.959  0.004

In [28]:
#Describe agents
res[["worst_agent", "top_agent", "best_team_average", "random_team_average", "N_agents", "l"]].groupby(["N_agents", "l"]).describe().loc[:, (slice(None), ["mean", "std"])]

worst_agent        top_agent        best_team_average         \
                   mean    std      mean    std              mean    std   
N_agents l                                                                 
10       12      83.809  0.184    86.390  0.051            86.235  0.065   
         20      83.724  0.075    86.410  0.281            86.289  0.256   
20       12      84.392  1.116    86.634  0.954            86.454  0.979   
         20      83.339  0.151    86.575  0.215            86.354  0.222   

            random_team_average         
                           mean    std  
N_agents l                              
10       12              85.196  0.394  
         20              85.106  0.145  
20       12              85.602  1.037  
         20              85.109  0.176